Try to learn (1) as a piecewise constant curve

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats
import os
from itertools import chain
set_matplotlib_formats('pdf', 'svg')

from deepthermal.FFNN_model import fit_FFNN, FFNN, init_xavier
from deepthermal.validation import create_subdictionary_iterator, k_fold_cv_grid, add_dictionary_iterators
from deepthermal.plotting import plot_result, plot_model_1d

from deep_reparametrization.plotting import plot_reparametrization
from deep_reparametrization.reparametrization import (
    get_elastic_metric_loss,
    compute_loss_reparam,
    get_elastic_error_func,
)
from deep_reparametrization.ResNET import ResNET
from deep_reparametrization.helpers import get_pc_curve, get_pl_curve
import experiments.curves as c1


In [3]:
########
DIR = "../figures/curve_1_pc/"
SET_NAME = "curve_pc_1"
PATH_FIGURES = os.path.join(DIR,SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########


FOLDS = 1
N = 128 # training points internal

loss_func = get_elastic_metric_loss(r=get_pc_curve(c1.r,N), constrain_cost=1e3, verbose=False)
penalty_free_loss_func = get_elastic_metric_loss(r=get_pc_curve(c1.r,N), constrain_cost=0, verbose=False)

MODEL_PARAMS = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": [ "relu"],
    "n_hidden_layers": [2],#,8,16,64],
    "neurons": [32]
}

# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "model": [FFNN],
}
TRAINING_PARAMS = {
    "batch_size": [N],
    "regularization_param": [0],
    "compute_loss": [compute_loss_reparam],
    "loss_func": [loss_func],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["strong_wolfe"],
    "num_epochs": [10],
    "learning_rate": [ 1],
}


In [4]:
# create iterators
model_params_iter= create_subdictionary_iterator(MODEL_PARAMS)

model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(TRAINING_PARAMS_EXPERIMENT, product=False)
exp_training_params_iter = add_dictionary_iterators(training_exp_iter, training_params_iter)

In [5]:
# Load data
x_train = torch.linspace(0, 1, N, requires_grad=True).unsqueeze(1)
q_pl = get_pc_curve(c1.q,N)
q_train = q_pl(x_train.detach())

data = TensorDataset(x_train, q_train)


Do the actual training

In [6]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_FFNN,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=3
)


Running model (trial=0, mod=0, k=0):
Parameters: ({'input_dimension': 1, 'output_dimension': 1, 'activation': 'relu', 'n_hidden_layers': 2, 'neurons': 32, 'model': <class 'deepthermal.FFNN_model.FFNN'>}, {'batch_size': 128, 'regularization_param': 0, 'compute_loss': <function compute_loss_reparam at 0x124e64790>, 'loss_func': <function get_elastic_metric_loss.<locals>.elastic_metric_loss at 0x125035790>, 'optimizer': 'strong_wolfe', 'num_epochs': 10, 'learning_rate': 1})
################################  0  ################################
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Training Loss:  134.50137329
Final training Loss:  134.50137329

Running model (trial=1, mod=1, k=0):
Parameters: ({'input_dimension': 1, 'output_dimension': 1, 'activation': 'relu', 'n_hidden

In [7]:
# plotting

disc_points = x_train.detach()

plot_kwargs = {
    "x_test": disc_points,
    "x_train": disc_points,
    "y_train": c1.ksi(disc_points),
    "x_axis": "t",
    "y_axis": "$\\varphi(t)$",
    "compare_label" : "analytical solution (smooth)"
}
plot_result(
    path_figures=PATH_FIGURES,
    plot_name=SET_NAME,
    **cv_results,
    plot_function=plot_model_1d,
    function_kwargs=plot_kwargs,
)